In [12]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
import datetime
import tensorflow as tf

In [13]:
root_path = "/home/sridharpatil047/transfer_learning/"
img_dir = root_path+"data_final"
df = pd.read_csv(root_path+'labels_final.csv')

#df = df.head(50) ## Comment this later
df.head()

,path,label
0,imagesv/v/o/h/voh71d00/509132755+-2755.tif,3
1,imagesl/l/x/t/lxt19d00/502213303.tif,3
2,imagesx/x/e/d/xed05a00/2075325674.tif,2
3,imageso/o/j/b/ojb60d00/517511301+-1301.tif,3
4,imagesq/q/z/k/qzk17e00/2031320195.tif,7


In [14]:
## converting labels to string because flow_from_dataframe requires labels to be str
df['label'] = df.label.astype(str)

df.shape

(48000, 2)

In [15]:
BATCH_SIZE = 50
EPOCHS =5
IMG_SIZE = (224, 224)

In [16]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.33,
                                                         preprocessing_function=tf.keras.applications.vgg16.preprocess_input)

train_generator = datagen.flow_from_dataframe(df,directory=img_dir, x_col='path', y_col='label',subset='training', 
                                              class_mode='categorical', target_size=IMG_SIZE,
                                              batch_size=BATCH_SIZE)

valid_generator = datagen.flow_from_dataframe(df,directory=img_dir, x_col='path', y_col='label',subset='validation',
                                              class_mode='categorical', target_size=IMG_SIZE,
                                              batch_size=BATCH_SIZE)

Found 32160 validated image filenames belonging to 16 classes.
Found 15840 validated image filenames belonging to 16 classes.


In [17]:
train_generator.n, valid_generator.n

(32160, 15840)

In [18]:
train_generator.image_shape, valid_generator.image_shape

((224, 224, 3), (224, 224, 3))

# Model3

In [19]:
## code inspired from here: https://www.tensorflow.org/tutorials/images/transfer_learning
model3_path = root_path+'models/model3/'

In [20]:
## source: https://www.tensorflow.org/tutorials/images/transfer_learning#freeze_the_convolutional_base
IMG_SHAPE = IMG_SIZE + (3,)
model_VGG16 = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=IMG_SHAPE)

## Making last 6 layers trainable and rest non-trainable
for layer in model_VGG16.layers[:13]:
    layer.trainable = False

for layer in model_VGG16.layers:
    print(layer, layer.trainable)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f4b099ec310> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f4b099ec8d0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f4b0bde8d90> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f4b0bdee390> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f4bc29e3150> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f4bc295a610> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f4b099f1c90> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f4bc29bb150> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f4b099f6b50> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f4b099fb4d0> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f4b09987bd0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 

In [21]:
tf.keras.backend.clear_session()

model = tf.keras.Sequential()
model.add(model_VGG16)

model.add(tf.keras.layers.Conv2D(1024, (7,7), padding='valid', activation='sigmoid'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Conv2D(512, (1,1), padding='valid', activation='sigmoid'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Conv2D(16, (1,1), padding='valid', activation='sigmoid'))
model.add(tf.keras.layers.Flatten())

base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['acc'])

log_dir = model3_path + datetime.datetime.now().strftime("%m%d%Y-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(train_generator,
          epochs=EPOCHS,
          validation_data=valid_generator,
         callbacks=[tensorboard_callback])
model.save(model3_path + 'model3.h5')

Epoch 1/5
644/644 [==============================] - 7024s 11s/step - loss: 1.4948 - acc: 0.5804 - val_loss: 1.0376 - val_acc: 0.6898
Epoch 2/5
644/644 [==============================] - 6830s 11s/step - loss: 0.9048 - acc: 0.7384 - val_loss: 0.7961 - val_acc: 0.7626
Epoch 3/5
644/644 [==============================] - 6838s 11s/step - loss: 0.7208 - acc: 0.7900 - val_loss: 0.7586 - val_acc: 0.7753
Epoch 4/5
644/644 [==============================] - 6829s 11s/step - loss: 0.5888 - acc: 0.8289 - val_loss: 0.7032 - val_acc: 0.7942
Epoch 5/5
644/644 [==============================] - 6829s 11s/step - loss: 0.4803 - acc: 0.8619 - val_loss: 0.6867 - val_acc: 0.8040
